In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, labels, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'geum_pull'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 15)

In [12]:
#run_num = range(len(folder))
run_num = [0]
#if file_parameters['watershed'] == 'nak':
#    run_num = range(len(folder))
#else:
#    run_num = [0]

__GAIN_TRAINING__ = True

real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx'], ['대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx'], ['장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx']]
Epoch 1/2000
1/1 [==============================] - 0s 195ms/step - gen_loss: 120.3329 - disc_loss: 0.7306 - rmse: 1.1991 - val_loss: 0.8317
Epoch 2/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 88.4351 - disc_loss: 0.5353 - rmse: 0.8454 - val_loss: 0.7555
Epoch 3/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 64.6101 - disc_loss: 0.5014 - rmse: 0.7132 - val_loss: 0.7178
Epoch 4/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 80.0509 - disc_loss: 0.4348 - rmse: 0.9362 - val_loss: 0.6647
Epoch 5/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 54.3645 - disc_loss: 0.3909 - rmse: 0.6806 - val_loss: 0.7051
Epoch 6/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 11.6082 - disc_loss: 0.3176 - rmse: 0.5329 - val_loss: 0.2607
Epoch 58/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 15.3967 - disc_loss: 0.3347 - rmse: 0.4681 - val_loss: 0.4390
Epoch 59/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7912 - disc_loss: 0.3279 - rmse: 0.2812 - val_loss: 0.3933
Epoch 60/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2817 - disc_loss: 0.3396 - rmse: 0.2735 - val_loss: 0.2322
Epoch 61/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6598 - disc_loss: 0.3237 - rmse: 0.2968 - val_loss: 0.5184
Epoch 62/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1153 - disc_loss: 0.3399 - rmse: 0.3036 - val_loss: 0.3170
Epoch 63/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6696 - disc_loss: 0.3288 - rmse: 0.2626 - val_loss: 0.4904
Epoch 64/2000
1/1 [====================

1/1 [==============================] - 0s 21ms/step - gen_loss: 6.4702 - disc_loss: 0.2979 - rmse: 0.3153 - val_loss: 0.4110
Epoch 116/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.5858 - disc_loss: 0.2731 - rmse: 0.3921 - val_loss: 0.5769
Epoch 117/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 17.4545 - disc_loss: 0.2827 - rmse: 0.5361 - val_loss: 0.2754
Epoch 118/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.4341 - disc_loss: 0.2892 - rmse: 0.3940 - val_loss: 0.3958
Epoch 119/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1941 - disc_loss: 0.2769 - rmse: 0.2441 - val_loss: 0.2462
Epoch 120/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2022 - disc_loss: 0.2777 - rmse: 0.2523 - val_loss: 0.2152
Epoch 121/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8961 - disc_loss: 0.2581 - rmse: 0.2229 - val_loss: 0.2616
Epoch 122/2000
1/1 [============

1/1 [==============================] - 0s 20ms/step - gen_loss: 14.5638 - disc_loss: 0.2520 - rmse: 0.4729 - val_loss: 0.2393
Epoch 174/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6712 - disc_loss: 0.2590 - rmse: 0.2423 - val_loss: 0.2841
Epoch 175/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1717 - disc_loss: 0.2487 - rmse: 0.2071 - val_loss: 0.1992
Epoch 176/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.2428 - disc_loss: 0.2613 - rmse: 0.4570 - val_loss: 0.2958
Epoch 177/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4289 - disc_loss: 0.2678 - rmse: 0.2132 - val_loss: 0.2708
Epoch 178/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1068 - disc_loss: 0.2516 - rmse: 0.2439 - val_loss: 0.5622
Epoch 179/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0256 - disc_loss: 0.2721 - rmse: 0.3912 - val_loss: 0.2179
Epoch 180/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7305 - disc_loss: 0.2589 - rmse: 0.1882 - val_loss: 0.2363
Epoch 232/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6586 - disc_loss: 0.2140 - rmse: 0.2396 - val_loss: 0.2256
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4727 - disc_loss: 0.2229 - rmse: 0.2665 - val_loss: 0.2349
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1351 - disc_loss: 0.2286 - rmse: 0.2081 - val_loss: 0.3684
Epoch 235/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.7472 - disc_loss: 0.2517 - rmse: 0.1949 - val_loss: 0.2127
Epoch 236/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2577 - disc_loss: 0.2244 - rmse: 0.1767 - val_loss: 0.2129
Epoch 237/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1650 - disc_loss: 0.2406 - rmse: 0.1814 - val_loss: 0.2614
Epoch 238/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7757 - disc_loss: 0.1809 - rmse: 0.2040 - val_loss: 0.1780
Epoch 986/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.1489 - disc_loss: 0.1682 - rmse: 0.1549 - val_loss: 0.1937
Epoch 987/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9924 - disc_loss: 0.1756 - rmse: 0.1983 - val_loss: 0.2689
Epoch 988/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 18.4868 - disc_loss: 0.1611 - rmse: 0.3681 - val_loss: 0.1916
Epoch 989/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9168 - disc_loss: 0.1757 - rmse: 0.1734 - val_loss: 0.1546
Epoch 990/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.4759 - disc_loss: 0.1686 - rmse: 0.2173 - val_loss: 0.2193
Epoch 991/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.6858 - disc_loss: 0.1781 - rmse: 0.1878 - val_loss: 0.1775
Epoch 992/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 11.4574 - disc_loss: 0.1766 - rmse: 0.2480 - val_loss: 0.2073
Epoch 1044/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1513 - disc_loss: 0.1866 - rmse: 0.2128 - val_loss: 0.1906
Epoch 1045/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.0564 - disc_loss: 0.1669 - rmse: 0.2832 - val_loss: 0.1821
Epoch 1046/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1222 - disc_loss: 0.1543 - rmse: 0.1935 - val_loss: 0.1951
Epoch 1047/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5345 - disc_loss: 0.1721 - rmse: 0.2244 - val_loss: 0.2163
Epoch 1048/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.8804 - disc_loss: 0.1683 - rmse: 0.1513 - val_loss: 0.2124
Epoch 1049/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0361 - disc_loss: 0.1770 - rmse: 0.2675 - val_loss: 0.2083
Epoch 1050/2000
1/1 [======

1/1 [==============================] - 0s 22ms/step - gen_loss: 2.6628 - disc_loss: 0.1676 - rmse: 0.1838 - val_loss: 0.2087
Epoch 1102/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3338 - disc_loss: 0.1647 - rmse: 0.1639 - val_loss: 0.4209
Epoch 1103/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 10.4615 - disc_loss: 0.1788 - rmse: 0.2172 - val_loss: 0.1820
Epoch 1104/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8299 - disc_loss: 0.1751 - rmse: 0.2023 - val_loss: 0.1798
Epoch 1105/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9181 - disc_loss: 0.1664 - rmse: 0.1805 - val_loss: 0.4882
Epoch 1106/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.1119 - disc_loss: 0.1649 - rmse: 0.1607 - val_loss: 0.4043
Epoch 1107/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.1834 - disc_loss: 0.1674 - rmse: 0.1827 - val_loss: 0.1770
Epoch 1108/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1711 - disc_loss: 0.1703 - rmse: 0.1916 - val_loss: 0.3653
Epoch 1160/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.0288 - disc_loss: 0.1616 - rmse: 0.3216 - val_loss: 0.1926
Epoch 1161/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6470 - disc_loss: 0.1729 - rmse: 0.2795 - val_loss: 0.1695
Epoch 1162/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.5771 - disc_loss: 0.1722 - rmse: 0.3130 - val_loss: 0.4172
Epoch 1163/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.9185 - disc_loss: 0.1720 - rmse: 0.2762 - val_loss: 0.5220
Epoch 1164/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.5424 - disc_loss: 0.1702 - rmse: 0.3060 - val_loss: 0.1932
Epoch 1165/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5246 - disc_loss: 0.1663 - rmse: 0.2108 - val_loss: 0.1836
Epoch 1166/2000
1/1 [======

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6971 - disc_loss: 0.1719 - rmse: 0.2744 - val_loss: 0.2035
Epoch 1218/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5602 - disc_loss: 0.1601 - rmse: 0.1909 - val_loss: 0.4106
Epoch 1219/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4716 - disc_loss: 0.1649 - rmse: 0.1672 - val_loss: 0.1705
Epoch 1220/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7142 - disc_loss: 0.1651 - rmse: 0.1938 - val_loss: 0.2947
Epoch 1221/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3511 - disc_loss: 0.1670 - rmse: 0.1913 - val_loss: 0.1988
Epoch 1222/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.9443 - disc_loss: 0.1666 - rmse: 0.1526 - val_loss: 0.1601
Epoch 1223/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3474 - disc_loss: 0.1647 - rmse: 0.1946 - val_loss: 0.1720
Epoch 1224/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [59]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'toc'
print(rnn_target_column)



(26304, 36)
toc


In [60]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (18412, 36) val_df.shape :  (5261, 36) test_df.shape: (2631, 36)


In [61]:
#fake_df.shape[0]/8760

In [62]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [63]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [64]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/geum_pull/toc/


In [65]:
#rnn_epochs, rnn_in_setps
__RNN_TRAINING__ = True
rnn_continue_train = False
rnn_epochs = 15
rnn_steps_per_epoch = 5

## 모델 학습

In [66]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, 
    epochs=rnn_epochs, steps_per_epoch = rnn_steps_per_epoch,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=rnn_continue_train)

Epoch 1/15
5/5 [==============================] - 5s 985ms/step - loss: 0.6765 - mean_absolute_error: 0.6674 - nse: -0.3683 - val_loss: 0.1157 - val_mean_absolute_error: 0.2669 - val_nse: 0.1362
Epoch 2/15
5/5 [==============================] - 5s 916ms/step - loss: 0.3542 - mean_absolute_error: 0.4563 - nse: 0.3407 - val_loss: 0.1497 - val_mean_absolute_error: 0.3091 - val_nse: -0.1186
Epoch 3/15
5/5 [==============================] - 5s 914ms/step - loss: 0.2497 - mean_absolute_error: 0.3845 - nse: 0.5382 - val_loss: 0.1758 - val_mean_absolute_error: 0.3419 - val_nse: -0.3150
Epoch 4/15
5/5 [==============================] - 5s 911ms/step - loss: 0.1889 - mean_absolute_error: 0.3343 - nse: 0.6452 - val_loss: 0.1567 - val_mean_absolute_error: 0.3237 - val_nse: -0.1660
Epoch 5/15
5/5 [==============================] - 5s 916ms/step - loss: 0.1569 - mean_absolute_error: 0.3037 - nse: 0.6891 - val_loss: 0.1402 - val_mean_absolute_error: 0.3144 - val_nse: -0.0478
Epoch 6/15
5/5 [=========

In [67]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

save/geum/models/geum_pull/toc/
year : 2016 ~ 2018
run :  [0]
target :  toc   4
length :  26304
train=7, val=1, test=2
all_nse :  [0.81523615]
all_pbias :  [1.2938286]
all_mae :  0.10127315
all_rmse :  0.14013667
all_Rs :  0.821715849934904
all_R :  0.90648544


In [68]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
train_nse :  [0.86270046]
train_pbias :  [0.31430382]
train_mae :  0.086894184
train_rmse :  0.124727026
train_Rs :  0.8640467878349831
train_R :  0.9295412


In [69]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
val_nse :  [0.2113195]
val_pbias :  [5.1592207]
val_mae :  0.1374634
val_rmse :  0.16939417
val_Rs :  0.49731958924100184
val_R :  0.7052089


In [70]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
test_nse :  [0.3768983]
test_pbias :  [2.21017]
test_mae :  0.109563805
test_rmse :  0.1511168
test_Rs :  0.6608115569957249
test_R :  0.81290317


In [29]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:

sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
        training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1

